# Water Quality Prediction System

This notebook implements a comprehensive machine learning pipeline for predicting water quality based on various chemical and physical parameters. The system uses multiple algorithms, performs hyperparameter tuning, and provides a complete analysis of water quality classification.

## Dataset Features:
- **Temperature**: Water temperature
- **Turbidity**: Water clarity measure (cm)
- **Dissolved Oxygen**: DO levels (mg/L)
- **BOD**: Biological Oxygen Demand (mg/L)
- **pH**: Water pH level
- **Ammonia**: Ammonia concentration (mg/L)
- **Nitrite**: Nitrite concentration (mg/L)

## Target Classes:
- **0**: Excellent water quality
- **1**: Good water quality  
- **2**: Poor water quality

## 1. Import Required Libraries

Import all necessary libraries for data processing, machine learning, and visualization. This includes scikit-learn for ML algorithms, pandas for data manipulation, and matplotlib/seaborn for plotting.

In [25]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.utils import resample
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO

## 2. Data Loading and Preprocessing

The `load_and_preprocess_data` function handles:
- Loading CSV data from file or string
- Converting European decimal format (comma to dot)
- Renaming columns for better readability
- Optional dataset balancing to handle class imbalance
- Displaying original and balanced class distributions

In [26]:
def load_and_preprocess_data(data_string_or_file, balance_dataset=True, target_samples_per_class=1400):
    """Load and preprocess the water quality data with optional balancing"""
    
    # Read the data - modify this based on your actual data source
    if isinstance(data_string_or_file, str) and len(data_string_or_file) < 1000:
        # If it's a file path
        try:
            df = pd.read_csv(data_string_or_file)
        except:
            # If it's a data string
            df = pd.read_csv(StringIO(data_string_or_file))
    else:
        # If it's a data string
        df = pd.read_csv(StringIO(data_string_or_file))
    
    # Handle comma decimal separators (European format)
    feature_columns = ['Temp', 'Turbidity (cm)', 'DO(mg/L)', 'BOD (mg/L)', 
                      'pH', 'Ammonia (mg L-1 )', 'Nitrite (mg L-1 )']
    
    for col in feature_columns:
        # Remove quotes and replace comma with dot for decimal separator
        df[col] = df[col].astype(str).str.replace('"', '').str.replace(',', '.').astype(float)
    
    # Separate features and target
    X = df[feature_columns]
    y = df['Water Quality']
    
    # Feature names for better interpretation
    feature_names = ['Temperature', 'Turbidity', 'Dissolved_Oxygen', 'BOD', 
                    'pH', 'Ammonia', 'Nitrite']
    X.columns = feature_names
    
    print("Original dataset distribution:")
    print(y.value_counts().sort_index())
    print()
    
    # Balance the dataset if requested
    if balance_dataset:
        X_balanced, y_balanced = balance_classes(X, y, target_samples_per_class)
        print(f"Balanced dataset distribution (target: {target_samples_per_class} samples per class):")
        print(y_balanced.value_counts().sort_index())
        print()
        return X_balanced, y_balanced, feature_names
    
    return X, y, feature_names

## 3. Class Balancing Function

The `balance_classes` function addresses class imbalance by:
- Resampling each class to have equal number of samples
- Using bootstrap sampling (with replacement) when necessary
- Shuffling the balanced dataset
- Maintaining data integrity while ensuring fair representation of all classes

In [27]:
def balance_classes(X, y, target_samples_per_class=1400):
    """Balance the dataset by resampling to equal number of samples per class"""
    
    # Combine features and target
    df = pd.concat([X, y], axis=1)
    
    # Separate by class
    class_0 = df[df['Water Quality'] == 0]  # Excellent
    class_1 = df[df['Water Quality'] == 1]  # Good  
    class_2 = df[df['Water Quality'] == 2]  # Poor
    
    # Resample each class to target number
    class_0_resampled = resample(class_0, 
                                replace=len(class_0) < target_samples_per_class,
                                n_samples=target_samples_per_class,
                                random_state=42)
    
    class_1_resampled = resample(class_1,
                                replace=len(class_1) < target_samples_per_class,
                                n_samples=target_samples_per_class,
                                random_state=42)
    
    class_2_resampled = resample(class_2,
                                replace=len(class_2) < target_samples_per_class,
                                n_samples=target_samples_per_class,
                                random_state=42)
    
    # Combine resampled classes
    df_balanced = pd.concat([class_0_resampled, class_1_resampled, class_2_resampled])
    
    # Shuffle the balanced dataset
    df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)
    
    # Separate features and target
    X_balanced = df_balanced.drop('Water Quality', axis=1)
    y_balanced = df_balanced['Water Quality']
    
    return X_balanced, y_balanced

## 4. Model Evaluation and Comparison

The `evaluate_models` function implements comprehensive model comparison:
- **Models tested**: Random Forest, Gradient Boosting, SVM, Logistic Regression, K-Nearest Neighbors
- **Evaluation metrics**: Cross-validation accuracy, test accuracy, per-class F1-scores
- **Stratified sampling**: Maintains class distribution in train/test splits
- **Feature scaling**: Applied appropriately for distance-based algorithms
- **Performance reporting**: Detailed classification reports for each model

In [28]:
def evaluate_models(X, y):
    """Evaluate multiple machine learning models with proper stratification"""
    
    # Split the data with stratification to maintain class balance
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                        random_state=42, 
                                                        stratify=y)
    
    print(f"Training set distribution:")
    print(y_train.value_counts().sort_index())
    print(f"Test set distribution:")
    print(y_test.value_counts().sort_index())
    print()
    
    # Scale the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Define models to test
    models = {
        'Random Forest': RandomForestClassifier(random_state=42, n_estimators=100),
        'Gradient Boosting': GradientBoostingClassifier(random_state=42, n_estimators=100),
        'SVM': SVC(random_state=42, probability=True),  # Enable probability for better analysis
        'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
        'K-Nearest Neighbors': KNeighborsClassifier()
    }
    
    results = {}
    
    print("Model Evaluation Results:")
    print("=" * 60)
    
    # Use stratified k-fold for cross-validation
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    for name, model in models.items():
        # Use scaled data for SVM, Logistic Regression, and KNN
        if name in ['SVM', 'Logistic Regression', 'K-Nearest Neighbors']:
            X_train_use = X_train_scaled
            X_test_use = X_test_scaled
        else:
            X_train_use = X_train
            X_test_use = X_test
        
        # Train the model
        model.fit(X_train_use, y_train)
        
        # Make predictions
        y_pred = model.predict(X_test_use)
        
        # Calculate accuracy
        accuracy = accuracy_score(y_test, y_pred)
        
        # Cross-validation score with stratification
        cv_scores = cross_val_score(model, X_train_use, y_train, cv=skf, scoring='accuracy')
        
        results[name] = {
            'model': model,
            'accuracy': accuracy,
            'cv_mean': cv_scores.mean(),
            'cv_std': cv_scores.std(),
            'scaler': scaler if name in ['SVM', 'Logistic Regression', 'K-Nearest Neighbors'] else None,
            'y_pred': y_pred
        }
        
        print(f"{name}:")
        print(f"  Test Accuracy: {accuracy:.4f}")
        print(f"  CV Score: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")
        
        # Class-wise performance
        report = classification_report(y_test, y_pred, output_dict=True, 
                                     target_names=['Excellent', 'Good', 'Poor'])
        print(f"  Per-class F1-scores:")
        for class_name in ['Excellent', 'Good', 'Poor']:
            print(f"    {class_name}: {report[class_name]['f1-score']:.4f}")
        print()
    
    return results, X_test, y_test

## 5. Hyperparameter Optimization

The `optimize_best_model` function fine-tunes the best performing model:
- **Grid Search**: Systematic search through hyperparameter combinations
- **Stratified Cross-Validation**: Ensures representative sampling during optimization
- **Model-specific parameters**: Tailored parameter grids for each algorithm type
- **Class balancing**: Includes class_weight parameter for imbalanced data handling
- **Pipeline creation**: Combines preprocessing and model for deployment

In [29]:
def optimize_best_model(X, y, best_model_name):
    
    # Use stratified split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42, stratify=y)
    
    # Use stratified k-fold for grid search
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    if best_model_name == 'Random Forest':
        model = RandomForestClassifier(random_state=42)
        param_grid = {
            'n_estimators': [100, 200, 300],
            'max_depth': [None, 10, 20, 30],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
            'class_weight': [None, 'balanced']  # Handle any remaining class imbalance
        }
        X_train_use = X_train
        X_test_use = X_test
        scaler = None
        
    elif best_model_name == 'Gradient Boosting':
        model = GradientBoostingClassifier(random_state=42)
        param_grid = {
            'n_estimators': [100, 200],
            'max_depth': [3, 5, 7],
            'learning_rate': [0.01, 0.1, 0.2],
            'subsample': [0.8, 1.0]
        }
        X_train_use = X_train
        X_test_use = X_test
        scaler = None
        
    else:  # For SVM, Logistic Regression, etc.
        scaler = StandardScaler()
        X_train_use = scaler.fit_transform(X_train)
        X_test_use = scaler.transform(X_test)
        
        if best_model_name == 'SVM':
            model = SVC(random_state=42, probability=True)
            param_grid = {
                'C': [0.1, 1, 10, 100],
                'gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1],
                'kernel': ['rbf', 'poly', 'sigmoid'],
                'class_weight': [None, 'balanced']
            }
        elif best_model_name == 'Logistic Regression':
            model = LogisticRegression(random_state=42, max_iter=1000)
            param_grid = {
                'C': [0.01, 0.1, 1, 10, 100],
                'penalty': ['l1', 'l2'],
                'solver': ['liblinear', 'saga'],
                'class_weight': [None, 'balanced']
            }
        elif best_model_name == 'K-Nearest Neighbors':
            model = KNeighborsClassifier()
            param_grid = {
                'n_neighbors': [3, 5, 7, 9, 11],
                'weights': ['uniform', 'distance'],
                'metric': ['euclidean', 'manhattan', 'minkowski']
            }
    
    # Perform grid search with stratified CV
    grid_search = GridSearchCV(model, param_grid, cv=skf, scoring='accuracy', n_jobs=-1, verbose=1)
    grid_search.fit(X_train_use, y_train)
    
    # Best model
    best_model = grid_search.best_estimator_
    
    # Predictions
    y_pred = best_model.predict(X_test_use)

    # Create pipeline if scaler is needed
    if scaler is not None:
        best_model = Pipeline([
            ('scaler', scaler),
            ('classifier', best_model)
        ])
    
    print(f"Best model performance:")
    print(f"Cross-validation score: {grid_search.best_score_:.4f}")
    print(f"Test accuracy: {accuracy_score(y_test, y_pred):.4f}")
    
    return best_model, scaler, X_test, y_test, y_pred, grid_search.best_params_

## 6. Feature Importance Analysis

The `analyze_feature_importance` function provides insights into which water quality parameters are most influential:
- **Tree-based models**: Extracts feature importance scores
- **Ranking**: Orders features by their predictive power
- **Interpretation**: Helps understand which chemical/physical parameters matter most for water quality prediction

In [30]:
def analyze_feature_importance(model, feature_names, model_name):
    """Analyze feature importance for tree-based models"""
    
    if hasattr(model, 'feature_importances_'):
        importance_df = pd.DataFrame({
            'feature': feature_names,
            'importance': model.feature_importances_
        }).sort_values('importance', ascending=False)
        
        print(f"\nFeature Importance ({model_name}):")
        print("=" * 40)
        for _, row in importance_df.iterrows():
            print(f"{row['feature']}: {row['importance']:.4f}")
        
        return importance_df
    else:
        print(f"\nFeature importance not available for {model_name}")
        return None

## 7. Load Dataset

Load the water quality dataset from CSV file for processing.

In [31]:
with open('./Water-Clarity-DS.csv', 'r', encoding='utf-8') as f:
    sample_data = f.read()

## 8. Additional Imports for Model Deployment

Import libraries needed for model serialization and deployment preparation.

In [32]:
import joblib
import json
from sklearn.pipeline import Pipeline

## 9. Prediction Function Creation

The `create_prediction_function` creates a user-friendly interface for making predictions:
- **Input validation**: Ensures proper data formatting
- **Preprocessing**: Applies same scaling used during training
- **Prediction**: Returns both class prediction and probability scores
- **User-friendly output**: Translates numeric classes to meaningful labels

In [33]:
def create_prediction_function(best_model, scaler, feature_names):
    """Create a function for making new predictions"""
    
    def predict_water_quality(temp, turbidity, do, bod, ph, ammonia, nitrite):
        """
        Predict water quality based on input parameters
        
        Parameters:
        - temp: Temperature
        - turbidity: Turbidity (cm)
        - do: Dissolved Oxygen (mg/L)
        - bod: BOD (mg/L)
        - ph: pH
        - ammonia: Ammonia (mg/L)
        - nitrite: Nitrite (mg/L)
        
        Returns:
        - prediction: 0 (excellent), 1 (good), 2 (poor)
        - probability: probability for each class
        """
        
        # Create input DataFrame with proper feature names
        input_data = pd.DataFrame([[temp, turbidity, do, bod, ph, ammonia, nitrite]], 
                                 columns=feature_names)
        
        # Scale if necessary
        if scaler is not None:
            input_data_scaled = pd.DataFrame(scaler.transform(input_data), 
                                           columns=feature_names)
            input_for_prediction = input_data_scaled
        else:
            input_for_prediction = input_data
        
        # Make prediction
        prediction = best_model.predict(input_for_prediction)[0]
        
        # Get probabilities if available
        if hasattr(best_model, 'predict_proba'):
            probabilities = best_model.predict_proba(input_for_prediction)[0]
        else:
            probabilities = None
        
        quality_labels = {0: 'Excellent', 1: 'Good', 2: 'Poor'}
        
        return {
            'prediction': prediction,
            'quality': quality_labels[prediction],
            'probabilities': probabilities
        }
    
    return predict_water_quality

## 10. Complete Machine Learning Pipeline

This cell executes the complete ML pipeline:

### Process Flow:
1. **Data Loading**: Load and preprocess the water quality dataset
2. **Class Balancing**: Balance classes to 1400 samples each
3. **Model Comparison**: Evaluate 5 different ML algorithms
4. **Best Model Selection**: Choose model with highest CV score
5. **Hyperparameter Tuning**: Optimize the best model parameters
6. **Performance Evaluation**: Generate detailed classification reports
7. **Feature Analysis**: Analyze which parameters are most important
8. **Model Deployment**: Save model and create prediction function
9. **Example Prediction**: Demonstrate how to use the trained model

### Output Includes:
- Dataset statistics and class distributions
- Model performance comparisons
- Confusion matrix and classification metrics
- Feature importance rankings
- Saved model files for deployment

In [35]:
# Main execution
if __name__ == "__main__":
    # Load and preprocess data with balancing
    # Replace 'your_data_file.csv' with your actual file path or data string
    X, y, feature_names = load_and_preprocess_data(sample_data, balance_dataset=True, target_samples_per_class=1400)
    
    print("Dataset shape:", X.shape)
    print("Feature names:", feature_names)
    print("Final target distribution:")
    print(y.value_counts().sort_index())
    print()
    
    # Evaluate models
    results, X_test, y_test = evaluate_models(X, y)
    
    # Find best model based on cross-validation score for better generalization
    best_model_name = max(results.keys(), key=lambda x: results[x]['cv_mean'])
    print(f"Best performing model: {best_model_name}")
    print(f"Best CV score: {results[best_model_name]['cv_mean']:.4f}")
    print(f"Test accuracy: {results[best_model_name]['accuracy']:.4f}")
    print()
    
    # Optimize best model
    print("Optimizing best model with hyperparameter tuning...")
    best_model, scaler, X_test_opt, y_test_opt, y_pred_opt, best_params = optimize_best_model(X, y, best_model_name)
    
    print("Best parameters:", best_params)
    print()
    
    # Detailed classification report
    print("Detailed Classification Report:")
    print("=" * 50)
    print(classification_report(y_test_opt, y_pred_opt, target_names=['Excellent', 'Good', 'Poor']))
    
    # Confusion matrix
    print("Confusion Matrix:")
    cm = confusion_matrix(y_test_opt, y_pred_opt)
    print("    Predicted")
    print("    Exc  Good Poor")
    print("Exc", cm[0])
    print("Good", cm[1])  
    print("Poor", cm[2])
    print()
    
    # Feature importance analysis
    importance_df = analyze_feature_importance(best_model, feature_names, best_model_name)
    
    # Create prediction function
    predict_func = create_prediction_function(best_model, scaler, feature_names)
    
    # Example prediction
    print("\nExample Prediction:")
    example_result = predict_func(67.45, 10.13, 0.208, 7.474, 4.752, 0.286, 4.355)
    print(f"Quality: {example_result['quality']} (Class: {example_result['prediction']})")
    if example_result['probabilities'] is not None:
        print("Probabilities - Excellent: {:.3f}, Good: {:.3f}, Poor: {:.3f}".format(
            example_result['probabilities'][0], 
            example_result['probabilities'][1], 
            example_result['probabilities'][2]))
    
    print("\nModel is ready for deployment!")
    print("Dataset has been balanced to 1400 samples per class")
    print("Use predict_func(temp, turbidity, do, bod, ph, ammonia, nitrite) to make predictions")


     # ===== NEW: Save artifacts for deployment =====
    # 1. Save model
    model_filename = "water_quality_model.pkl"
    joblib.dump(best_model, model_filename)
    
    # 2. Save feature names
    with open('feature_names.json', 'w') as f:
        json.dump(feature_names, f)
    
    # 3. Save class labels mapping
    class_labels = {0: 'Excellent', 1: 'Good', 2: 'Poor'}
    with open('class_labels.json', 'w') as f:
        json.dump(class_labels, f)

    print(f"\nModel saved to {model_filename}")
    print("Feature names and class labels saved")


Original dataset distribution:
Water Quality
0    1400
1    1400
2    1500
Name: count, dtype: int64

Balanced dataset distribution (target: 1400 samples per class):
Water Quality
0    1400
1    1400
2    1400
Name: count, dtype: int64

Dataset shape: (4200, 7)
Feature names: ['Temperature', 'Turbidity', 'Dissolved_Oxygen', 'BOD', 'pH', 'Ammonia', 'Nitrite']
Final target distribution:
Water Quality
0    1400
1    1400
2    1400
Name: count, dtype: int64

Training set distribution:
Water Quality
0    1120
1    1120
2    1120
Name: count, dtype: int64
Test set distribution:
Water Quality
0    280
1    280
2    280
Name: count, dtype: int64

Model Evaluation Results:
Random Forest:
  Test Accuracy: 0.9536
  CV Score: 0.9583 (+/- 0.0114)
  Per-class F1-scores:
    Excellent: 0.9771
    Good: 0.9542
    Poor: 0.9284

Random Forest:
  Test Accuracy: 0.9536
  CV Score: 0.9583 (+/- 0.0114)
  Per-class F1-scores:
    Excellent: 0.9771
    Good: 0.9542
    Poor: 0.9284

Gradient Boosting:
  Test

## 11. Results and Model Deployment

This section is for additional analysis, visualizations, or testing of the deployed model. The trained model has been saved to disk along with feature names and class labels for easy deployment.